<a href="https://colab.research.google.com/github/CatIsOutOfTheBag/PetSQL/blob/main/SQL4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Джойны

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
users = pd.DataFrame({'id': [1,2,3],
                      'name': ['gleb','jon snow', 'tyrion']})
items = pd.DataFrame({'user_id': [1,3],
                      'item_name': ['hleb','gold'],
                      'value': [5,100]})

In [3]:
users

,id,name
0,1,gleb
1,2,jon snow
2,3,tyrion


In [4]:
items

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100


In [5]:
con = sqlite3.connect('db')
users.to_sql('users', con, index=False, if_exists='replace')
items.to_sql('items', con, index=False, if_exists='replace')

In [6]:
def select(sql):
  return pd.read_sql(sql, con)

In [7]:
sql = '''select * from users t'''

In [8]:
select(sql)

,id,name
0,1,gleb
1,2,jon snow
2,3,tyrion


Задача:
Подтянуть предметы, которыми владеют персонажи

LEFT JOIN - джоин без потери строк

In [9]:
sql = '''select t.*, i.item_name 
from users t left join items i
on t.id = i.user_id'''

select(sql)

,id,name,item_name
0,1,gleb,hleb
1,2,jon snow,None
2,3,tyrion,gold


INNER JOIN - останутся только те строки, по которым что-то нашлось во второй таблице

In [10]:
sql = '''select t.*, i.item_name 
from users t inner join items i
on t.id = i.user_id'''

select(sql)

,id,name,item_name
0,1,gleb,hleb
1,3,tyrion,gold


In [11]:
# В результате джойнов могут появиться дубликаты - например, если у персонажа не один предмет
# тогда будет новая строка с новым предметом

users = pd.DataFrame({'id': [1,2,3],
                      'name': ['gleb','jon snow', 'tyrion'],
                      'victory': [2,10,1]})
items = pd.DataFrame({'user_id': [1,3,3],
                      'item_name': ['hleb','gold','wine'],
                      'value': [5,100,20]})

users.to_sql('users', con, index=False, if_exists='replace')
items.to_sql('items', con, index=False, if_exists='replace')

In [12]:
sql = '''select * from users t'''

select(sql)

,id,name,victory
0,1,gleb,2
1,2,jon snow,10
2,3,tyrion,1


In [13]:
sql = '''select * from items t'''

select(sql)

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100
2,3,wine,20


In [14]:
# сджойним новые таблицы
# порядок столбцов можно просто задать руками

sql = '''select t.*, i.item_name, i.value, i.user_id
from users t left join items i
on t.id = i.user_id'''

select(sql)

,id,name,victory,item_name,value,user_id
0,1,gleb,2,hleb,5.0,1.0
1,2,jon snow,10,None,NaN,NaN
2,3,tyrion,1,gold,100.0,3.0
3,3,tyrion,1,wine,20.0,3.0


In [15]:
# видим, что тирион и его предметы - две строки таблцы. 
# теперь просто просуммировать победы не получится

In [16]:
t = select(sql)

In [17]:
t['victory'].sum()

14

In [18]:
sql = '''select sum(t.victory) from users t'''
select(sql)

,sum(t.victory)
0,13


Правило:
1. Всегда проверять тоталы
2. Агрегировать таблицы, в которых есть дубликаты перед тем, как их джойнить

In [19]:
# сагрегируем таблицу items по user перед тем, как джойнить ее
sql = '''select * from items t'''

select(sql)

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100
2,3,wine,20


In [20]:
sql = '''select 
t.user_id,
count(t.item_name) as items_cnt,
sum(t.value) as value_sum

from items t
group by t.user_id
'''

select(sql)

,user_id,items_cnt,value_sum
0,1,1,5
1,3,2,120


In [21]:
# теперь делаем джойн
# для избавления от NaN-ов используем coalesce

sql = '''
with grouped_items as
(
select t.user_id, count(t.item_name) as items_cnt, sum(t.value) as value_sum
from items t
group by t.user_id
)

select t.*, coalesce(i.items_cnt, 0) as items_cnt, coalesce(i.value_sum, 0) as value_sum, i.user_id
from users t left join grouped_items i
on t.id = i.user_id
'''
select(sql)

,id,name,victory,items_cnt,value_sum,user_id
0,1,gleb,2,1,5,1.0
1,2,jon snow,10,0,0,NaN
2,3,tyrion,1,2,120,3.0


В постгресс еще реализован right join и full join

In [22]:
# можно сымитировать фулджойн с помощью юнион (лефт джойн и имитации райт джойн)
sql = '''select t.*, i.* 
from users t left join items i
on t.id = i.user_id

union

select u.*, t.* 
from items t left join users u
on t.user_id = u.id
'''

select(sql)

,id,name,victory,user_id,item_name,value
0,1,gleb,2,1.0,hleb,5.0
1,2,jon snow,10,NaN,None,NaN
2,3,tyrion,1,3.0,gold,100.0
3,3,tyrion,1,3.0,wine,20.0


Выгрузка информации о конкретных клиентах с помощью inner join

https://drive.google.com/file/d/1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR/view?usp=sharing

In [23]:
# Заливаем данные
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 64.3MB/s]


In [24]:
# Сохраняем данные в датафрейм
df = pd.read_csv('/content/german_credit_augmented.csv')

In [25]:
# Контролируем тип ДАТА, так как он заливается просто как текст
df['contract_dt'] = pd.to_datetime(df['contract_dt'], format='%Y-%m-%d %H:%M:%S')

In [26]:
# Создаем файл с базой данных на гугл-диске
# перед этим шагом монтируем гугл диск
con = sqlite3.connect('/content/drive/My Drive/german_credit.db')

In [27]:
# Сохраняем таюлицу в базу
df.to_sql('german_credit', con, index=False, if_exists='replace')


In [28]:
clients = pd.DataFrame({'client_id': [200,45],
                        'data': [1,2]})

In [29]:
clients

,client_id,data
0,200,1
1,45,2


In [30]:
clients.to_sql('clients', con, index=False, if_exists='replace')

In [31]:
sql = '''select * from clients t'''
select(sql)

,client_id,data
0,200,1
1,45,2


In [32]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [33]:
sql = '''
select t.*, cln.data 
from german_credit t join clients cln
on t.client_id = cln.client_id
'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id,data
0,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200,1
1,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45,2


Генерация заготовок под отчет с помощью inner join

In [34]:
# например, сгенерировать для каждого юзера все месяца года
sql = '''select 1 as user_id'''

In [35]:
sql = '''select 1 as user_id
union all
select 2 as user_id
union all
select 3 as user_id
'''

In [36]:
select(sql)

,user_id
0,1
1,2
2,3


In [37]:
sql = '''select date('2021-03-01') as month
union all
select date('2021-04-01') as month
'''
select(sql)

,month
0,2021-03-01
1,2021-04-01


In [38]:
# сделаем заготовку. На каждого юзера - каждый месяц

Кросс джоин

In [39]:
sql = '''
with users as
(
select 1 as user_id
union all
select 2 as user_id
union all
select 3 as user_id
),

month as
(
select date('2021-03-01') as month
union all
select date('2021-04-01') as month
)

select * from users t
join month m on 1=1

'''

# join month m on 1=1 - константное условие

In [40]:
select(sql)

,user_id,month
0,1,2021-03-01
1,2,2021-03-01
2,3,2021-03-01
3,1,2021-04-01
4,2,2021-04-01
5,3,2021-04-01


# Задача о транзакциях

Посчитать ежемесячное количество транзакций и их сумму в каждом месяце

https://drive.google.com/file/d/1ksjv4EsLItdn9rYA4oxislish-c4eE4D/view?usp=sharing

In [41]:
! gdown --id 1ksjv4EsLItdn9rYA4oxislish-c4eE4D

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ksjv4EsLItdn9rYA4oxislish-c4eE4D
To: /content/german_credit_augmented_transactions.csv
100% 134k/134k [00:00<00:00, 64.1MB/s]


In [42]:
transactions = pd.read_csv('/content/german_credit_augmented_transactions.csv')

In [43]:
transactions.head()

,dt,client_id,amount
0,2008-04-06 11:54:47,950,161.38
1,2007-07-28 00:00:19,418,35.34
2,2008-03-14 20:43:54,131,146.50
3,2007-12-18 13:03:24,353,119.21
4,2007-11-09 05:18:30,849,105.24


In [44]:
transactions.to_sql('client_transactions',con,index=False,if_exists='replace')

In [45]:
# количество транзакций
sql = '''select count(*) from client_transactions t limit 5'''
select(sql)

,count(*)
0,4275


In [46]:
# первый день месяца для каждой даты
sql = '''select t.*, date(t.dt, 'start of month') as month from client_transactions t'''
select(sql)

,dt,client_id,amount,month
0,2008-04-06 11:54:47,950,161.38,2008-04-01
1,2007-07-28 00:00:19,418,35.34,2007-07-01
2,2008-03-14 20:43:54,131,146.50,2008-03-01
3,2007-12-18 13:03:24,353,119.21,2007-12-01
4,2007-11-09 05:18:30,849,105.24,2007-11-01
...,...,...,...,...
4270,2007-08-18 04:05:05,185,10063.07,2007-08-01
4271,2007-06-04 15:23:32,375,156.91,2007-06-01
4272,2007-12-06 21:34:06,418,10053.82,2007-12-01
4273,2008-04-19 17:30:07,409,10050.35,2008-04-01


In [47]:
# сгруппируем по месяцам
sql = '''select  
date(t.dt, 'start of month') as month,
count(*) as trans_count,
sum(t.amount) as amount_sum

from client_transactions t
group by 1
order by 1
'''

select(sql)

,month,trans_count,amount_sum
0,2007-05-01,338,450912.77
1,2007-06-01,379,551664.83
2,2007-07-01,304,494134.50
3,2007-08-01,255,426903.23
4,2007-10-01,332,634846.49
5,2007-11-01,389,500420.98
6,2007-12-01,364,561449.89
7,2008-01-01,413,630137.22
8,2008-02-01,228,337043.47
9,2008-03-01,309,425599.09


И вот внимание! В таблице нет сентября. А при отсутствии сентября, среднее и другие показатели окажутся ложными.

Поэтому всегда нужно создавать последовательность месяцев/годов/дней/тд, если эта последовательность константна

Нужно делать вот так:

In [48]:
# Находим минимальную и максимальную дату в таблице
min_date = '''select date(min(t.dt), 'start of month') as min_date from client_transactions t'''
max_date = '''select date(max(t.dt), 'start of month') as max_date from client_transactions t'''

In [49]:
select(min_date)

,min_date
0,2007-05-01


In [50]:
select(max_date)

,max_date
0,2008-06-01


In [51]:
# сгенерируем последовательность месяцев от min до max
import datetime as dt
start = select(min_date)['min_date'].values[0]
end = select(max_date)['max_date'].values[0]

months = pd.DataFrame({'month': pd.date_range(start=start, end=end, freq='MS')})

In [52]:
months

,month
0,2007-05-01
1,2007-06-01
2,2007-07-01
3,2007-08-01
4,2007-09-01
5,2007-10-01
6,2007-11-01
7,2007-12-01
8,2008-01-01
9,2008-02-01


In [53]:
# зальем сгенерированную последовательность месяцев в базу
months.to_sql('months', con, index=False, if_exists='replace')

In [54]:
sql = '''select date(t.month, 'start of month') as month from months t'''
select(sql)

,month
0,2007-05-01
1,2007-06-01
2,2007-07-01
3,2007-08-01
4,2007-09-01
5,2007-10-01
6,2007-11-01
7,2007-12-01
8,2008-01-01
9,2008-02-01


In [55]:
# сделаем left join к сгенерированным месяца таблице с данными
sql = '''
with month_gen as
(
select date(t.month, 'start of month') as month from months t
),

month_trans as
(
select  
date(t.dt, 'start of month') as month,
count(*) as trans_count,
sum(t.amount) as amount_sum

from client_transactions t
group by 1
order by 1
)

select mg.month, 
coalesce(mt.trans_count,0) as trans_count,
coalesce(mt.amount_sum,0) as amount_sum
from month_gen mg left join month_trans mt on mg.month=mt.month
'''

select(sql)

,month,trans_count,amount_sum
0,2007-05-01,338,450912.77
1,2007-06-01,379,551664.83
2,2007-07-01,304,494134.50
3,2007-08-01,255,426903.23
4,2007-09-01,0,0.00
5,2007-10-01,332,634846.49
6,2007-11-01,389,500420.98
7,2007-12-01,364,561449.89
8,2008-01-01,413,630137.22
9,2008-02-01,228,337043.47


# Задача о пользователях

сколько каждый пользователь тратит по месяцам

In [56]:
# все пользователи лежат в таюлице german_credit, даже неактивные
# возьмем все уникальные id
sql = '''select distinct t.client_id from german_credit t'''
select(sql)

,client_id
0,210
1,929
2,200
3,45
4,358
...,...
995,624
996,181
997,730
998,557


In [57]:
# необходимо сгенерировать даты на каждого пользователя
# это будет сделано кросс-джойном

sql = '''
with month_gen as
(
select date(t.month, 'start of month') as month from months t
),

clients as
(
  select distinct t.client_id from german_credit t
),

clients_month as
(
select t.month, c.client_id from month_gen t
join clients c on 1 = 1
-- проверка по одному пользователю where c.client_id = 929
)

select * from clients_month t 
'''

select(sql)

,month,client_id
0,2007-05-01,210
1,2007-05-01,929
2,2007-05-01,200
3,2007-05-01,45
4,2007-05-01,358
...,...,...
13995,2008-06-01,624
13996,2008-06-01,181
13997,2008-06-01,730
13998,2008-06-01,557


In [58]:
# для каждого пользователя посчитаем количество транзакций и сумму в таблице транзакций
sql = '''select  
date(t.dt, 'start of month') as month,
t.client_id,
count(*) as trans_count,
sum(t.amount) as amount_sum

from client_transactions t
group by 1,2  --теперь группировка по месяцам и клиентам
order by 1'''

select(sql)

,month,client_id,trans_count,amount_sum
0,2007-05-01,101,1,149.67
1,2007-05-01,107,2,217.50
2,2007-05-01,110,1,1081.32
3,2007-05-01,111,1,139.97
4,2007-05-01,113,1,1051.29
...,...,...,...,...
3551,2008-06-01,983,1,10050.37
3552,2008-06-01,987,1,192.48
3553,2008-06-01,992,1,-610.21
3554,2008-06-01,996,1,121.28


In [59]:
# теперь этот запрос можно добавить в отчет - CTE

sql = '''
with month_gen as -- сгенерированные месяца
(
select date(t.month, 'start of month') as month from months t
),

clients as -- все id клиентов из большой таблицы
(
  select distinct t.client_id from german_credit t
),

clients_month as -- сгенерированные месяца и клиенты вместе
(
select t.month, c.client_id from month_gen t
join clients c on 1 = 1
-- проверка по одному пользователю where c.client_id = 929
),

trans_month as -- транзакции по месяцам
(
select  
date(t.dt, 'start of month') as month,
t.client_id,
count(*) as trans_count,
sum(t.amount) as amount_sum

from client_transactions t
group by 1,2  --теперь группировка по месяцам и клиентам
order by 1
)

-- теперь к таблице, где есть все месяца и все клиенты можно приджойнить то, что
-- имеется и никого не пропустить

select t.*, tm.trans_count, tm.amount_sum from clients_month t 
left join trans_month tm on (t.month = tm.month) and (t.client_id = tm.client_id)
--where t.client_id = 929 -- на одного клиента
'''

select(sql)

,month,client_id,trans_count,amount_sum
0,2007-05-01,210,1.0,68.84
1,2007-05-01,929,1.0,156.40
2,2007-05-01,200,NaN,NaN
3,2007-05-01,45,NaN,NaN
4,2007-05-01,358,NaN,NaN
...,...,...,...,...
13995,2008-06-01,624,1.0,170.43
13996,2008-06-01,181,1.0,133.53
13997,2008-06-01,730,NaN,NaN
13998,2008-06-01,557,1.0,66.02


In [60]:
# посчитать, сколько активных клиентов было каждый месяц
# теперь этот запрос можно добавить в отчет - CTE

sql = '''
with month_gen as -- сгенерированные месяца
(
select date(t.month, 'start of month') as month from months t
),

clients as -- все id клиентов из большой таблицы
(
  select distinct t.client_id from german_credit t
),

clients_month as -- сгенерированные месяца и клиенты вместе
(
select t.month, c.client_id from month_gen t
join clients c on 1 = 1
-- проверка по одному пользователю where c.client_id = 929
),

trans_month as -- транзакции по месяцам
(
select  
date(t.dt, 'start of month') as month,
t.client_id,
count(*) as trans_count,
sum(t.amount) as amount_sum

from client_transactions t
group by 1,2  --теперь группировка по месяцам и клиентам
order by 1
),

report as
(
-- теперь к таблице, где есть все месяца и все клиенты можно приджойнить то, что
-- имеется и никого не пропустить

select t.*, tm.trans_count, tm.amount_sum,
1 as user, -- хардкод для удобного суммирования
case when tm.trans_count > 0 then 1 else 0 end as active

from clients_month t 
left join trans_month tm on (t.month = tm.month) and (t.client_id = tm.client_id)
)


-- сделаем сводник на основе большой таблицы report
select t.month, sum(t.user) as user_cnt, sum(t.active) as active_cnt, sum(t.amount_sum) as amount_sum
from report t
group by t.month'''

select(sql)

,month,user_cnt,active_cnt,amount_sum
0,2007-05-01,1000,288,450912.77
1,2007-06-01,1000,297,551664.83
2,2007-07-01,1000,259,494134.50
3,2007-08-01,1000,222,426903.23
4,2007-09-01,1000,0,NaN
5,2007-10-01,1000,283,634846.49
6,2007-11-01,1000,323,500420.98
7,2007-12-01,1000,287,561449.89
8,2008-01-01,1000,325,630137.22
9,2008-02-01,1000,204,337043.47


In [61]:
# проверим, бьются ли тоталы по деньгам
t = select(sql)

In [62]:
t['amount_sum'].sum()

6548980.619999999

In [63]:
sql = '''select sum(t.amount) from client_transactions t'''

select(sql)

,sum(t.amount)
0,6548980.62


In [64]:
# все отлично, отчет сделан правильно

Таблицу можно джойнить саму с собой

In [70]:
# найдем нарастающую сумму из таблицы (1,2,3 -> 1,3,6)
df = pd.DataFrame({'date': pd.to_datetime(['2007-01-01', '2007-01-02', '2007-01-03'], format = '%Y-%m-%d'),
                   'revenue': [1,2,3]})

In [71]:
df

,date,revenue
0,2007-01-01,1
1,2007-01-02,2
2,2007-01-03,3


In [72]:
df.to_sql('revenue', con, index=False, if_exists='replace')

In [73]:
sql = '''select * from revenue t'''
select(sql)

,date,revenue
0,2007-01-01 00:00:00,1
1,2007-01-02 00:00:00,2
2,2007-01-03 00:00:00,3


In [76]:
sql = '''select * from revenue t
join revenue d on d.date <= t.date
'''
select(sql)

,date,revenue,date,revenue
0,2007-01-01 00:00:00,1,2007-01-01 00:00:00,1
1,2007-01-02 00:00:00,2,2007-01-01 00:00:00,1
2,2007-01-02 00:00:00,2,2007-01-02 00:00:00,2
3,2007-01-03 00:00:00,3,2007-01-01 00:00:00,1
4,2007-01-03 00:00:00,3,2007-01-02 00:00:00,2
5,2007-01-03 00:00:00,3,2007-01-03 00:00:00,3


In [78]:
# осталось сделать группировку
sql = '''select t.date, t.revenue, sum(d.revenue) as revemu_sum from revenue t
join revenue d on d.date <= t.date
group by t.date, t.revenue
'''
select(sql)

,date,revenue,revemu_sum
0,2007-01-01 00:00:00,1,1
1,2007-01-02 00:00:00,2,3
2,2007-01-03 00:00:00,3,6
